In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.mongodb.spark:mongo-spark-connector_2.11:2.3.1 pyspark-shell'
os.environ['SPARK_HOME'] = './spark-2.4.4-bin-hadoop2.7'
os.environ['PYSPARK_PYTHON']='/usr/lib/python3.5'
# !export SPARK_HOME=./spark-2.4.4-bin-hadoop2.7
import findspark
findspark.init()
findspark.find()
import time
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('BDA_A2_Mongodb').master('local') \
        .config("spark.mongodb.input.uri", "mongodb://127.0.0.1:27017/assignment2.table1") \
        .config("spark.mongodb.output.uri", "mongodb://127.0.0.1:27017/assignment2.table1") \
        .config('spark jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.1') \
        .getOrCreate()

df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()
df.printSchema()
df.createOrReplaceTempView('GHTORRENT_RECORDS')

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- access_key: string (nullable = true)
 |-- downloader_id: string (nullable = true)
 |-- logging_level: string (nullable = true)
 |-- request_status: string (nullable = true)
 |-- retrieval_stage: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- url: string (nullable = true)



In [3]:
spark2 = SparkSession.builder.appName('BDA_A2_Mongodb').master('local') \
        .config("spark.mongodb.input.uri", "mongodb://127.0.0.1:27017/assignment2.table2") \
        .config("spark.mongodb.output.uri", "mongodb://127.0.0.1:27017/assignment2.table2") \
        .config('spark jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.1') \
        .getOrCreate()
df2 = spark2.read.format("com.mongodb.spark.sql.DefaultSource").load()
df2.printSchema()
df2.createOrReplaceTempView('IMPORTANT_REPOS')

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- deleted: string (nullable = true)
 |-- forked_from: string (nullable = true)
 |-- id: string (nullable = true)
 |-- language: string (nullable = true)
 |-- name: string (nullable = true)
 |-- owner_id: string (nullable = true)
 |-- updated_at: string (nullable = true)
 |-- url: string (nullable = true)



In [4]:
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- access_key: string (nullable = true)
 |-- downloader_id: string (nullable = true)
 |-- logging_level: string (nullable = true)
 |-- request_status: string (nullable = true)
 |-- retrieval_stage: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- url: string (nullable = true)



In [5]:
df2.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- deleted: string (nullable = true)
 |-- forked_from: string (nullable = true)
 |-- id: string (nullable = true)
 |-- language: string (nullable = true)
 |-- name: string (nullable = true)
 |-- owner_id: string (nullable = true)
 |-- updated_at: string (nullable = true)
 |-- url: string (nullable = true)



In [10]:
query3 = """SELECT COUNT(*) FROM GHTORRENT_RECORDS WHERE LOGGING_LEVEL='WARN'"""
result3 = spark.sql(query3)
# print("Count the number of WARNing messages.")
result3.show()

+--------+
|count(1)|
+--------+
|  132158|
+--------+



In [11]:
query4 = '''SELECT COUNT(DISTINCT URL) 
               FROM GHTORRENT_RECORDS 
               WHERE RETRIEVAL_STAGE = 'api_client' AND URL != 'NULL' '''
result4 = spark.sql(query4)

In [12]:
result4.show()

+-------------------+
|count(DISTINCT URL)|
+-------------------+
|              78369|
+-------------------+



In [7]:
query5 = """SELECT COUNT(RETRIEVAL_STAGE), DOWNLOADER_ID
            FROM GHTORRENT_RECORDS 
            WHERE RETRIEVAL_STAGE='api_client' AND URL != 'NULL'
            GROUP BY DOWNLOADER_ID
            ORDER BY COUNT(RETRIEVAL_STAGE) DESC 
            LIMIT 10"""
result5 = spark.sql(query5)
result5.show()

+----------------------+-------------+
|count(RETRIEVAL_STAGE)|DOWNLOADER_ID|
+----------------------+-------------+
|                 85528|           13|
|                 19046|            4|
|                 18948|           18|
|                 18926|           10|
|                 18911|           40|
|                 18616|           39|
|                 18614|           38|
|                 18604|           47|
|                 18463|            1|
|                 18452|           24|
+----------------------+-------------+



In [ ]:
from pymongo import MongoClient
import json
from pprint import pprint

myclient = MongoClient('localhost', 27017)
myData = myclient['assignment2']

Table1 = myData['table1']
Table1.create_index('DOWNLOADER_ID')

In [9]:
query10 = '''
            SELECT COUNT(DISTINCT URL) 
            FROM GHTORRENT_RECORDS 
            WHERE DOWNLOADER_ID = '22' AND URL != 'NULL' '''

start_time = time.time()
result10 = spark.sql(query10)
end_time = time.time()

result10.show()
print("TimeTaken: {} ms".format(end_time-start_time))
# print(rows[0][0])

+-------------------+
|count(DISTINCT URL)|
+-------------------+
|               4390|
+-------------------+

TimeTaken: 0.5066001415252686 ms


In [8]:
Table1.drop_index('DOWNLOADER_ID_1')

In [4]:
query10 = '''
            SELECT COUNT(DISTINCT URL) 
            FROM GHTORRENT_RECORDS 
            WHERE DOWNLOADER_ID = '22' AND URL != 'NULL' '''

start_time = time.time()
result10 = spark.sql(query10)
end_time = time.time()

result10.show()
print("TimeTaken: {} ms".format(end_time-start_time))
# print(rows[0][0])

+-------------------+
|count(DISTINCT URL)|
+-------------------+
|               4390|
+-------------------+

TimeTaken: 1.1038885116577148 ms
